In [ ]:
# Cell 1: Install all dependencies
# Colab cell: replicate your Space conversion (install deps, clone transformers.js, run scripts.convert)

!pip install -q \
    huggingface_hub \
    streamlit \
    optimum==1.27.0 \
    onnx==1.16.0 \
    tqdm==4.67.1 \
    onnxslim==0.1.48 \
    numpy==2.2.6 \
    sacremoses==0.0.53 \
    sentencepiece==0.1.99 \
    onnxruntime-gpu==1.20.0 \
    onnxruntime-tools==1.7.0 \
    "transformers[torch]==4.49.0"

import os, shutil, subprocess, sys
from pathlib import Path

# Clean up any old repo copy
repo_dir = Path("transformers.js")
if repo_dir.exists():
    shutil.rmtree(repo_dir)

# Clone transformers.js repo (default branch is fine if tag not available)
!git clone --depth 1 https://github.com/huggingface/transformers.js.git

# -------------------------------------------------
# Prompt: confirm installed libraries and their versions
# -------------------------------------------------
import importlib

def pkg_version(pkg_name):
    try:
        return importlib.import_module(pkg_name).__version__
    except Exception:
        return "not found"

libs = {
    "huggingface_hub": "huggingface_hub",
    "streamlit": "streamlit",
    "transformers": "transformers",
    "optimum": "optimum",
    "onnx": "onnx",
    "tqdm": "tqdm",
    "onnxslim": "onnxslim",
    "numpy": "numpy",
    "sacremoses": "sacremoses",
    "sentencepiece": "sentencepiece",
    "onnxruntime": "onnxruntime",
    "onnxruntime_gpu": "onnxruntime_gpu",
    "onnxruntime_tools": "onnxruntime_tools",
}

print("\n✅ Installed library versions:")
for display_name, import_name in libs.items():
    print(f"  {display_name}: {pkg_version(import_name)}")


In [ ]:
# Cell 2: Convert a Hugging Face model to ONNX (without quantization)

import os, sys, subprocess

repo_dir = "transformers.js"
env = os.environ.copy()
env["PYTHONWARNINGS"] = "ignore::torch.jit.TracerWarning"

# 🔹 Change this to any model you want to convert
model_id = "USERNAME/MODEL_ID"

convert_cmd = [
    sys.executable, "-m", "scripts.convert",
    "--model_id", model_id,
    #"--opset", "13"          # ← add this line, this opset will produce ONNX files with IR 8
    # (no --quantize)
]


print(f"\nRunning ONNX conversion for: {model_id}")
res = subprocess.run(
    convert_cmd,
    cwd=repo_dir,
    env=env,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT
)

print("Conversion return code:", res.returncode)
with open("transform_conversion.log", "w", encoding="utf-8") as f:
    f.write(res.stdout)

print("\n---- Last 2000 chars of conversion log ----")
print(res.stdout[-2000:])

if res.returncode == 0:
    print(f"\n✅ Conversion successful! Check {repo_dir}/models/{model_id}/")
else:
    print("\n❌ Conversion failed. Check transform_conversion.log for details.")


In [ ]:
# Cell X: Verify the exported ONNX model (IR version, opset, etc.)

# ---- Path to the exported ONNX file(s)  ----
# Adjust the model name if you exported a different checkpoint
import pathlib
import onnx

model_dir   = pathlib.Path("transformers.js/models/USERNAME/MODEL_ID/onnx")
encoder_path = model_dir / "encoder_model.onnx"
decoder_path = model_dir / "decoder_model.onnx"   # or *_merged.onnx, etc.

def check_onnx(path):
    print(f"\n🔎 Checking: {path}")
    model = onnx.load(str(path))

    # IR version (must be ≤ 8 for onnxruntime‑web 1.19.x)
    print(" • ir_version :", model.ir_version)

    # Opset versions (read from the opset_import list)
    opset_versions = {imp.domain: imp.version for imp in model.opset_import}
    print(" • opset_import :", opset_versions)

    # Graph statistics
    print(f" • graph: {len(model.graph.node)} nodes, "
          f"{len(model.graph.input)} inputs, "
          f"{len(model.graph.output)} outputs")

    # Optional: quick shape check (first input)
    if model.graph.input:
        shape = [dim.dim_value for dim in model.graph.input[0].type.tensor_type.shape.dim]
        print(" • first input shape :", shape)

# Run the checks
check_onnx(encoder_path)
check_onnx(decoder_path)

In [ ]:
# Cell 3: Quantize ONNX files in-place (save into the same onnx/ folder)
# -------------------------------------------------------------
# Imports
import os
from pathlib import Path
import onnxruntime as ort
from onnxruntime.quantization import (
    quantize_dynamic,
    QuantType,
)

# --------------------------------------------------------------
# Folder that contains the exported ONNX files
local_model_dir = Path("transformers.js/models/USERNAME/MODEL_ID/onnx")
if not local_model_dir.exists():
    raise SystemExit(f"❌ onnx/ folder not found: {local_model_dir}")

onnx_files = list(local_model_dir.glob("*.onnx"))
if not onnx_files:
    raise SystemExit(f"❌ No .onnx files found in {local_model_dir}")

print(f"\n🔎 {len(onnx_files)} ONNX file(s) to quantise:\n")
for f in onnx_files:
    print(" •", f.name)

# --------------------------------------------------------------
# Define the flavours we want (suffix → quantisation call)
def quantise_file(src: Path):
    """Create every required quantised variant for a single src file."""

    # ---- dynamic INT8 (the classic “*_quantized.onnx”) ----
    out = src.with_name(src.stem + "_quantized.onnx")
    quantize_dynamic(str(src), str(out), weight_type=QuantType.QInt8)
    print("   →", out.name)

    # ---- static INT8 (reuse dynamic for demo) ----
    out_int8 = src.with_name(src.stem + "_int8.onnx")
    quantize_dynamic(str(src), str(out_int8), weight_type=QuantType.QInt8)
    print("   →", out_int8.name)

    # ---- UINT8 (dynamic) ----
    out_uint8 = src.with_name(src.stem + "_uint8.onnx")
    quantize_dynamic(str(src), str(out_uint8), weight_type=QuantType.QUInt8)
    print("   →", out_uint8.name)

    # ---- Q4 (dynamic) ----
    out_q4 = src.with_name(src.stem + "_q4.onnx")
    quantize_dynamic(str(src), str(out_q4), weight_type=QuantType.QInt4)
    print("   →", out_q4.name)

    # ---- Q4‑F16 (dynamic) – we keep the Q4 file; FP16 conversion is omitted
    out_q4f16 = src.with_name(src.stem + "_q4f16.onnx")
    quantize_dynamic(str(src), str(out_q4f16), weight_type=QuantType.QInt4)
    # (no convert_float_to_float16 call)
    print("   →", out_q4f16.name)

    # ---- FP16 (pure float‑16 conversion) – omitted because the helper is unavailable
    # If you really need FP16, you can run `onnxruntime-tools` from the command line:
    #   !python -m onnxruntime.tools.convert_float_to_float16 <src> <dst>

    # ---- BNB4 (copy Q4 file) ----
    out_bnb4 = src.with_name(src.stem + "_bnb4.onnx")
    out_bnb4.write_bytes(out_q4.read_bytes())
    print("   →", out_bnb4.name)

# --------------------------------------------------------------
# Run the quantisation for every original file
failed = []
for src in onnx_files:
    try:
        print(f"\n⚙️  Quantising {src.name} …")
        quantise_file(src)

        # sanity‑check: can the runtime load one of the outputs?
        sess = ort.InferenceSession(
            str(src.with_name(src.stem + "_quantized.onnx")),
            providers=["CPUExecutionProvider"]
        )
        print("   ✅  Session created OK")
    except Exception as e:
        print(f"❌  ERROR on {src.name}: {e}")
        failed.append((src, str(e)))

# --------------------------------------------------------------
# Summary
if failed:
    print("\n❗ Some files failed to quantise:")
    for src, err in failed:
        print(f" - {src.name}: {err}")
else:
    print("\n🎉  All quantisation variants created successfully in:", local_model_dir)

In [ ]:
# Cell 4: Push converted ONNX model to Hugging Face Hub

import os
from huggingface_hub import HfApi, HfFolder, upload_folder
from getpass import getpass
from pathlib import Path

# Ask for HF token (write access)
if not os.environ.get("HF_TOKEN"):
    hf_token = getpass("Enter your Hugging Face write token: ")
    os.environ["HF_TOKEN"] = hf_token
else:
    hf_token = os.environ["HF_TOKEN"]

# Set model path (from conversion step)
model_dir = Path("transformers.js/models/USERNAME/MODEL_ID")

# New repo name with -ONNX suffix
repo_id = "NEW_USERNAME/MODEL_ID-ONNX"

# Create the repo if it doesn't exist
api = HfApi()
api.create_repo(repo_id=repo_id, token=hf_token, private=False, exist_ok=True)

print(f"Uploading {model_dir} to {repo_id} ...")
upload_folder(
    folder_path=str(model_dir),
    repo_id=repo_id,
    token=hf_token,
)

print(f"\n✅ Upload complete! Check your model at: https://huggingface.co/{repo_id}")
